In [ ]:
!service elasticsearch start

In [1]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
#from retrieval_model import BertEncoder
from transformers import AdamW, TrainingArguments, get_linear_schedule_with_warmup
from torch import nn
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle
import os, json

In [2]:
train_dataset = load_from_disk("/opt/ml/data/new_train_dataset/train")
valid_dataset = load_from_disk("/opt/ml/data/train_dataset/validation")
wiki_dataset = pd.read_json('data/wikipedia_documents.json',orient='index') # wiki context
data_path = "data/"
context_path = "wikipedia_documents.json"
with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
    wiki = json.load(f)

# wiki_dataset = dict(
#     dict.fromkeys([v["text"] for v in wiki.values()])
# ) 


test_dataset = load_from_disk('/opt/ml/data/test_dataset/validation')


In [120]:
with open('data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))


In [3]:
valid_corpus = list(set(valid_dataset['context']))
train_corpus = list(set(train_dataset['context']))
wiki_corpus = list(set(wiki_dataset))
query = train_dataset['question']
test_query = test_dataset['question']

In [3]:
from elasticsearch import Elasticsearch, helpers
import pprint  

In [4]:
INDEX_NAME = "wiki_index"

INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "shingle" ],

          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }

  }
}

In [24]:
docs = [
    {
        '_index' : INDEX_NAME,
        '_id' : i,
        'content' : wiki_dataset.iloc[i]['text']
    }
    for i in range(wiki_dataset.shape[0])
]

In [21]:
docs = [
    {
        '_index' : INDEX_NAME,
        '_id' : i,
        'content' : contexts[i]
    }
    for i in range(len(contexts))
]

In [7]:
try:
    es.transport.close()
except:
    pass
es = Elasticsearch()

In [8]:
es.info()

/opt/conda/envs/bc/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': '3ef0ae13ffd8',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'QBJxOfIgS5mBwdLwFxXqyg',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [9]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

/tmp/ipykernel_10202/2824541921.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
/tmp/ipykernel_10202/2824541921.py:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_index'}

In [25]:
import time
start_time = time.time()
try:
    response = helpers.bulk(es, docs)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)
    
end_time = time.time()

print(end_time-start_time)


RESPONSE: (60613, [])
154.62924194335938


In [84]:
questions[1197] = 'SW 취약점 찾기 대회에서 수상한 해커그룹들이 찾은 보안 취약점의 특징은?'
questions[3] = '11~12 세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?'

In [99]:
error_questions = []
train_neg = [[] for _ in range(len(questions))]

for idx, (question, answer) in tqdm(enumerate(zip(questions, answers))):
    try:
        res = es.search(index=INDEX_NAME, q=question, size=30)
    except Exception as e:
        error_questions.append([e, question])
    for hits in res['hits']['hits']:
        train_neg[idx].append(hits['_id'])

3952it [03:49, 17.20it/s]


In [103]:
for idx1 in range(len(train_neg)):
    for idx2 in range(train_neg[0]):
        train_neg[idx1][idx2] = int(train_neg[idx1][idx2])

In [122]:
with open("elastic_train_neg", "wb") as file:
    pickle.dump(train_neg, file)

In [41]:
def get_hit_rate(questions, answers, size):
    error_questions = []
    hit_cnt = 0
    for question, answer in zip(questions, answers):
        try:
            res = es.search(index=INDEX_NAME, q=question, size=size)
        except Exception as e:
            error_questions.append([e, question])
        for hit in res['hits']['hits']:
            if answer['text'][0] in (hit['_source']['title']+hit['_source']['content']):
                hit_cnt += 1
                break
    print('size : '+str(size)+'\nhit rate', hit_cnt/len(questions))
    return error_questions
for size in [1, 5, 10, 20, 50, 100, 150, 200]:
    get_hit_rate(questions, answers, size)


size : 1
hit rate 0.7458333333333333
size : 5
hit rate 0.8833333333333333
size : 10
hit rate 0.9208333333333333
size : 20
hit rate 0.95
size : 50
hit rate 0.9708333333333333
size : 100
hit rate 0.9791666666666666
size : 150
hit rate 0.9791666666666666
size : 200
hit rate 0.9833333333333333


In [186]:
questions = test_dataset['question']
questions[0]

"유령'은 어느 행성에서 지구로 왔는가?"

In [209]:
def get_real_hit(questions, answers, size):
    error_questions = []
    hit_cnt = 0
    rank = [[] for _ in range(len(questions))]
    
    for idx, question in enumerate(questions):
        res = es.search(index=INDEX_NAME, q=question, size=size)
        
        for hit in res['hits']['hits']:
            rank[idx].append(int(hit['_id']))
            if int(hit['_id']) >= 56737:
                print('error')
                print(hit['_id'])
        
    return rank

rank = get_real_hit(questions, answers, size=20)


In [213]:
with open('data/rank.bin', "wb") as file:
    pickle.dump(rank, file)

In [218]:
questions[0]

"유령'은 어느 행성에서 지구로 왔는가?"